In [ ]:
from telethon.sync import TelegramClient
from telethon.tl.types import MessageMediaPhoto
import os
import json
from datetime import datetime
import logging

In [ ]:

api_id = "your_api_id_here"  # Replace with your actual API ID
api_hash = "your_api_hash_here"  # Replace with your actual API Hash
client = TelegramClient('session', api_id, api_hash)

# Set up logging
logging.basicConfig(level=logging.INFO)

async def scrape_channel(channel_usernames, limit=50):
    await client.start()
    for channel_username in channel_usernames:
        logging.info(f"Scraping messages from {channel_username}")
        
        messages = await client.get_messages(channel_username, limit=limit)
        os.makedirs(f"data/raw/telegram_messages/{datetime.now().strftime('%Y-%m-%d')}/{channel_username}", exist_ok=True)

        # Store messages in a list for JSON output
        message_data = []

        for i, message in enumerate(messages):
            if message.text:
                logging.info(f"{i+1}. {message.text}\n")
                message_data.append({"message": message.text})
            if message.media and isinstance(message.media, MessageMediaPhoto):
                file_path = await message.download_media(file=f"data/raw/telegram_messages/{datetime.now().strftime('%Y-%m-%d')}/{channel_username}/")
                logging.info(f"Saved image to: {file_path}")
                message_data.append({"image": file_path})
        
        # Save messages as JSON
        with open(f"data/raw/telegram_messages/{datetime.now().strftime('%Y-%m-%d')}/{channel_username}/messages.json", 'w') as f:
            json.dump(message_data, f, indent=4)

    await client.disconnect()

# Run this directly
await scrape_channel(["lobelia4cosmetics","tikvahpharma","CheMed123"])